In [486]:
import yaml
import sys
from pathlib import Path
from typing import Dict
import rdflib


In [487]:
def dict2yaml(path: str, data: Dict):
    with open(path, 'w') as yaml_f:
        yaml.safe_dump(data=data, stream=yaml_f)

In [488]:
# open a graph
graph = rdflib.Graph()
# load some data
graph.parse('https://raw.githubusercontent.com/tibonto/aeon/confIDent/aeon.ttl', format="ttl")  
# do a query
qres = graph.query(
    """
    PREFIX aeon: <https://github.com/tibonto/aeon#>
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    SELECT ?aeon_property ?WikidatURI ?WikidataLabel
    WHERE {
         ?aeon_property aeon:WikidataLabel ?WikidataLabel.
         OPTIONAL {?aeon_property aeon:WikidataURI ?WikidatURI.}
         
    }""")
     
print("the graph has %s statements" % len(qres))

mapping_dict = {}
for printout in qres:
    printout_dict = printout.asdict()
    #print (printout_dict)
    
    # make rdf literal to string and cut off base URI
    aeon_property = str(printout_dict.get('aeon_property'))
    aeon_property = aeon_property.replace("'","")
    aeon_property = aeon_property.replace("https://github.com/tibonto/aeon#","")
    
    # make rdf literal to string
    wikidatURI = str(printout_dict.get('WikidatURI'))
    wikidatURI = wikidatURI.replace("'","")
    
    # account for the fact that for multiple URI/Label pairs of the same aeon_property the relation is lost 
    # due to using two seperate annotation properties --> we should switch to storing both in one as JSON
    if wikidatURI == 'https://www.wikidata.org/wiki/Property:P5127':
        wikidataLabel = 'WikiCFP_conference_series_ID'
    elif wikidatURI == 'https://www.wikidata.org/wiki/Property:P646':
        wikidataLabel = 'Freebase_ID'
    else:
        wikidataLabel = str(printout_dict.get('WikidataLabel'))
        wikidataLabel = wikidataLabel.replace("'","")
        
    # account for the fact that multiple values for an aeon_property may exist
    # these pairs will be incorrect, if not added as possible cases in the above if/elif/else clause
    if aeon_property in mapping_dict.keys():
        x = mapping_dict[aeon_property].get('external_props')[0]
        print (x)
        if x != {'URI':wikidatURI, 'external_prop':wikidataLabel}:
            mapping_dict[aeon_property] = {'external_props':[x,
                                                            {'URI':wikidatURI, 'external_prop':wikidataLabel}]}
    else:
        mapping_dict[aeon_property] = {'external_props': [{'URI':wikidatURI, 'external_prop':wikidataLabel}]}

the graph has 20 statements
{'URI': 'https://www.wikidata.org/wiki/Property:P646', 'external_prop': 'Freebase_ID'}
{'URI': 'https://www.wikidata.org/wiki/Property:P646', 'external_prop': 'Freebase_ID'}
{'URI': 'https://www.wikidata.org/wiki/Property:P646', 'external_prop': 'Freebase_ID'}


In [489]:
print(len(mapping_dict))
mapping_dict.get('has_external_identifier')

17


{'external_props': [{'URI': 'https://www.wikidata.org/wiki/Property:P646',
   'external_prop': 'Freebase_ID'},
  {'URI': 'https://www.wikidata.org/wiki/Property:P5127',
   'external_prop': 'WikiCFP_conference_series_ID'}]}

In [490]:
dict2yaml('confident_mapping.yml', mapping_dict)